In [1]:
import requests
import pandas as pd
import json
token = 'm3lB5Q8zHn7wis2frICKXcmPb7yERygZnPypvcjuKPcye4IKkwdY'

In [3]:
base_url = 'https://api.lens.org/scholarly/search'


In [37]:
data = pd.read_csv('/Users/jmccarty/GitHub/urban_bipv_annotated_bib/data/lens_bipv_general.csv')
lens_id_list = data['Lens ID'].values.tolist()
lens_id_list[0:5]

['000-033-539-196-264',
 '000-044-578-669-216',
 '000-057-840-270-546',
 '000-066-246-955-002',
 '000-070-229-075-646']

In [31]:
lens_id_query = {
     "query": {
           "terms":{
                "lens_id": lens_id_list
           }
     }
}

lens_id_query2 = {
           "terms":{
                "lens_id": lens_id_list[0:500]
           }
     }

(
    ( 
    BIPV\* OR BAPV\* 
    OR 
    'building \b photovoltaic\*' 
    OR 
    photovoltaic* 
    OR 
    'photovoltaic thermal'
    ) 
AND 
    (urban 
    OR 
    (cities OR city)
    )
)

In [33]:
headers = {'Authorization': token, 'Content-Type': 'application/json'}
response = requests.post(base_url, json=lens_id_query, headers=headers)
if response.status_code != requests.codes.ok:
    print(response.status_code)
else:
    json_data = json.loads(response.text)
    print(len(json_data['data']))

20


In [36]:
json_data['data'][2]

{'lens_id': '022-295-678-613-126',
 'title': "ColiSense, today's sample today: A rapid on-site detection of β-D-Glucuronidase activity in surface water as a surrogate for E. coli",
 'publication_type': 'journal article',
 'year_published': 2015,
 'date_published': '2015-10-21T00:00:00.000000+00:00',
 'date_published_parts': [2015, 10, 21],
 'created': '2018-05-12T05:48:10.657000+00:00',
 'external_ids': [{'type': 'magid', 'value': '1871506881'},
  {'type': 'coreid', 'value': '95876006'},
  {'type': 'doi', 'value': '10.1016/j.talanta.2015.10.035'},
  {'type': 'pmid', 'value': '26653426'}],
 'patent_citations': [{'lens_id': '043-615-278-369-795'},
  {'lens_id': '052-064-542-832-514'}],
 'patent_citations_count': 2,
 'open_access': {'colour': 'green'},
 'authors': [{'first_name': 'Brendan',
   'last_name': 'Heery',
   'initials': 'B',
   'ids': [{'type': 'orcid', 'value': '0000-0002-8610-5238'},
    {'type': 'magid', 'value': '2924978454'}],
   'affiliations': [{'name': 'Dublin City Unive

In [30]:
import requests
import time
url = 'https://api.lens.org/scholarly/search'

# include fields
include = ["abstract", "lens_id"]
# request body with scroll time of 1 minute
request_body = {
     "query": lens_id_query2,
     "size": 100,
     "scroll":"1m",
     "include": include
}
headers = {'Authorization': token, 'Content-Type': 'application/json'}

# Recursive function to scroll through paginated results
def scroll(scroll_id):
  # Change the request_body to prepare for next scroll api call
  # Make sure to append the include fields to make faster response
  if scroll_id is not None:
    global request_body
    request_body = '''{"scroll_id": "%s", "include": %s}''' % (scroll_id, include)

  # make api request
  response = requests.post(url, json=request_body, headers=headers) 

  # If rate-limited, wait for n seconds and proceed the same scroll id
  # Since scroll time is 1 minutes, it will give sufficient time to wait and proceed
  if response.status_code == requests.codes.too_many_requests:
    time.sleep(8)
    scroll(scroll_id)
  # If the response is not ok here, better to stop here and debug it
  elif response.status_code != requests.codes.ok:
    print(response.json())
  # If the response is ok, do something with the response, take the new scroll id and iterate
  else:
    json = response.json()
    scroll_id = json['scroll_id'] # Extract the new scroll id from response
    scroll(scroll_id)
# start recursive scrolling
scrolled = scroll(scroll_id=None)

Good on MjAyMTI0;FGluY2x1ZGVfY29udGV4dF91dWlkDnF1ZXJ5VGhlbkZldGNoGBRxeW9KWFhvQnp3ZFFkOGQyTUc4TQAAAAAANt4KFlVOd1FabmN5Um02dmF1VkMtb1pKQ0EUcVNvSlhYb0J6d2RRZDhkMk1HOEwAAAAAADbeCBZVTndRWm5jeVJtNnZhdVZDLW9aSkNBFDBmVUpYWG9CNnlpQjVkZ1BNTjRMAAAAAAA7XUIWZG9sZVhDMURSeE94dHJSSndob1U5URQwdlVKWFhvQjZ5aUI1ZGdQTU40TAAAAAAAO11DFmRvbGVYQzFEUnhPeHRyUkp3aG9VOVEUMF9VSlhYb0I2eWlCNWRnUE1ONEwAAAAAADtdRBZkb2xlWEMxRFJ4T3h0clJKd2hvVTlRFHJDb0pYWG9CendkUWQ4ZDJNRzhNAAAAAAA23gsWVU53UVpuY3lSbTZ2YXVWQy1vWkpDQRRxaW9KWFhvQnp3ZFFkOGQyTUc4TAAAAAAANt4JFlVOd1FabmN5Um02dmF1VkMtb1pKQ0EUMVBVSlhYb0I2eWlCNWRnUE1ONEwAAAAAADtdRRZkb2xlWEMxRFJ4T3h0clJKd2hvVTlRFHFDb0pYWG9CendkUWQ4ZDJNRzhMAAAAAAA23gcWVU53UVpuY3lSbTZ2YXVWQy1vWkpDQRRyU29KWFhvQnp3ZFFkOGQyTUc4ZQAAAAAANt4MFlVOd1FabmN5Um02dmF1VkMtb1pKQ0EUcmlvSlhYb0J6d2RRZDhkMk1HOG8AAAAAADbeDRZVTndRWm5jeVJtNnZhdVZDLW9aSkNBFDFmVUpYWG9CNnlpQjVkZ1BNTjRMAAAAAAA7XUYWZG9sZVhDMURSeE94dHJSSndob1U5URQxdlVKWFhvQjZ5aUI1ZGdQTU40YwAAAAAAO11HFmRvbGVYQzFEUnhPeHRyUkp3aG9VOVEUcnlvSlhYb0J6d2RRZDhkMk1HOHEAAAA